#### In order to convert the csv files to DataFrames

In [ ]:
import pandas as pd
import numpy as np

customers_df = pd.read_csv('./data/raw/customers.csv')
customers_df.head(5)

In [ ]:
orders_df = pd.read_csv('./data/raw/orders.csv')
orders_df.head(5)

In [ ]:
products_df = pd.read_csv('./data/raw/products.csv')
products_df.head(5)

#### In order to clean the null values from the customers table

In [ ]:
# In order to confirm NULL values
print(customers_df[customers_df['name'].isna()])

In [ ]:
# In order to replace NULL values in the name column
customers_df['name'] = np.where(customers_df['name'].isna(), 'Guest', customers_df['name'])

In [ ]:
# In order to check that the data was transformed
guest_check = customers_df[customers_df['name'] == 'Guest']
guest_check

In [ ]:
# In order to confirm NULL values
print(customers_df[customers_df['email'].isna()])

In [ ]:
# In order to replace NULL values in the email column
customers_df['email'] = np.where(customers_df['email'].isna(), 'none_provided', customers_df['email'])
provision_check = customers_df[customers_df['email'] == 'none_provided']
provision_check

#### In order to clean the bad dates from the order table

In [ ]:
# In order to check non-date values in 'order_date'
convert = pd.to_datetime(orders_df['order_date'], errors='coerce')
bad_dates = orders_df[convert.isna()]
bad_dates

In [ ]:
# In order to replace 'not_a_date" and integer values and convert them to datetime
orders_df['order_date'] = orders_df['order_date'].replace('not_a_date', '1900-01-01')
orders_df['order_date'] = orders_df['order_date'].replace(20002, '2002-01-01')
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], errors='coerce')

null_check = orders_df['order_date'].isna().sum
null_check

#### In order to create the 8 dimension tables

In [ ]:
products_df

In [ ]:
# 1. customer_dim
customers_df

In [ ]:
# 2. product_category_dim
product_cat_df = pd.DataFrame({'product_category': products_df['product_category'].drop_duplicates()})
product_cat_df

In [ ]:
# 3. store_dim
store_df = pd.DataFrame({'store_id': orders_df['store_id'].drop_duplicates()})
store_df

In [ ]:
store_name = {1: 'nyc_store', 2: 'tampa_store', 3: 'seattle_store', 4: 'chicago_store'}
store_region = {1: 'East', 2: 'South', 3: 'West', 4: 'Midwest'}

store_df['store_name'] = store_df['store_id'].map(store_name)
store_df['store_region'] = store_df['store_id'].map(store_region)
store_df

In [ ]:
# 4. product_dim
product_df = products_df.drop(columns=['price'])
product_df

In [ ]:
# 5. order_date_dim
order_date_df = pd.DataFrame({'order_date': pd.to_datetime(orders_df['order_date'])})
order_date_df['order_month_id'] = order_date_df['order_date'].dt.month
order_date_df['order_year'] = order_date_df['order_date'].dt.year
order_date_df['order_quarter_id'] = order_date_df['order_date'].dt.quarter
order_date_df

In [ ]:
# 6. order_month_dim
months = {1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June", 7: "July", 8: "August", 9: "September", 10:"October", 11:"November", 12: "December"}
order_month_df = pd.DataFrame({'order_month_id': order_date_df['order_month_id'].drop_duplicates()})
order_month_df['order_month'] = order_month_df['order_month_id'].map(months)
order_month_df

In [ ]:
# 7. order_quarters_dim
quarters = {1: 'first_quarter', 2: 'second_quarter', 3: 'third_quarter', 4: 'fourth_quarter'}
order_quarter_df = pd.DataFrame({'order_quarter_id': order_date_df['order_quarter_id'].drop_duplicates()})
order_quarter_df['order_quarter'] = order_date_df['order_quarter_id'].map(quarters)
order_quarter_df

In [74]:
# 8. sales_facts_dim
# orders_df
product_prices_df = products_df[['product_id', 'price']]
sales_facts_df = pd.merge(orders_df, product_prices_df, on='product_id', how='left')
sales_facts_df

,order_id,cust_id,product_id,order_date,store_id,quantity,price
0,1,52,2,1988-05-19,3,2,92.48
1,2,70,4,1997-03-07,2,1,33.01
2,3,30,1,1985-07-31,4,3,17.35
3,4,475,7,2011-04-30,1,2,42.86
4,5,409,3,1981-06-19,3,3,69.05
...,...,...,...,...,...,...,...
995,996,45,2,2015-11-16,4,2,92.48
996,997,409,9,2016-01-09,1,2,21.26
997,998,40,1,1970-10-07,1,1,17.35
998,999,87,7,2022-03-15,3,2,42.86


In [ ]:
products_df
